In [1]:
import glob
import pandas as pd
import numpy as np
import pathlib

In [2]:
def is_empty(e):
    if e is None:
        return True
    if isinstance(e, str) and not e:
        return True
    if isinstance(e, float) and np.isnan(e):
        return True
    if hasattr(e, '__len__') and len(e) == 0:
        return True
    return False

def is_string(e):
    if isinstance(e, str) and e:
        return True
    return False

def is_number(e):
    if isinstance(e, float) and not np.isnan(e):
        return True
    if isinstance(e, str) and e:
        return (e
                .replace(" ", "")
                .replace("-", "")
                .replace("%", "")
                .replace(",", "")
                .isdigit()
               )
    return False

def identification_mask(pdf):
    def substitute(e):
        if is_empty(e):
            return None
        if is_number(e):
            return "n"
        if is_string(e):
            return "s"
    return pdf.applymap(substitute)

In [3]:
def identify_number_area(c):
    ic = identification_mask(c)
    dc = ic.copy()
    
    index_sum = float('inf')
    for i in range(ic.shape[0]):
        for j in range(ic.shape[1]):
            if 0 == (ic.iloc[i:, j:] == "s").sum().sum() and i + j < index_sum:
                index_sum = i + j
                max_i, max_j = i, j  

    return max_i, max_j

In [4]:
def squeeze_headers(c, number_area):
    
    def apply_transform(s):
        return s.apply(str).apply(lambda x: "" if x == "NaN" else x)
    
    sc = c.applymap(lambda x: "NaN" if isinstance(x, float) and np.isnan(x) else x)
    ie, je = number_area
    
    l1 = None
    for i in range(0, ie):
        if l1 is None:
            l1 = apply_transform(sc.iloc[0, :])
        else:
            l1 = l1.str.cat(apply_transform(sc.iloc[i]), sep=' ')
    if l1 is not None:
        sc = pd.concat([l1.to_frame().T, sc.drop(axis=0, index=range(0, ie))])
    
    l2 = None
    for i in range(0, je):
        if l2 is None:
            l2 = apply_transform(sc.iloc[:, 0])
        else:
            l2 = l2.str.cat(apply_transform(sc.iloc[:, i]), sep=' ')
    if l2 is not None:
        sc = pd.concat([l2.to_frame(), sc.drop(axis=1, columns=range(0, je))], axis=1)
    
    if l1 is not None:
        sc.iloc[0, :] = [f"*{i}" if x.strip() == "" else x for i, x in enumerate(sc.iloc[0, :])]
        sc.columns = sc.iloc[0, :]
        sc = sc.drop(axis=0, index=[0])
        
    sc = sc.set_index(sc.columns[0])
    
    return sc

def squeeze_columns(sc):
    columns_to_drop = []
    for c1, c2 in zip(sc.columns, sc.columns[1:]):
        all_is_nan = (sc[c1] == "NaN").all()
        if not isinstance(all_is_nan, pd.Series) and all_is_nan and c2[0] == "*":
            sc[c1] = sc[c2]
            columns_to_drop.append(c2)
            
    return sc.drop(columns=columns_to_drop)

def columns_to_numbers(sc):
    c

In [6]:
total_numbers = 0
for i, f in enumerate(glob.glob("../data/res/**/*.csv", recursive=True)):
    print("parsing", i, f)
    f = pathlib.Path(f)
    c = pd.read_csv(f, sep=',', thousands=' ', header=None, dtype=object, encoding='UTF-8')
#     display(c)
    ic = identification_mask(c)
#     display(ic)
    na = identify_number_area(c)
    print(na)
    sc = squeeze_headers(c, na)
#     display(sc)
    final = squeeze_columns(sc)
    
    try:
        final = final.applymap(lambda x: x.replace(" ", "").replace("%", "").replace(",", ".")).astype('float')
        total_numbers += final[final > 0].count().sum()
    except ValueError:
        pass
        
    display(final.iloc[:10,:])

    output_filename = "../data/nice/" + "".join(f.name.split(".")[:-1]) + ".xlsx"
    final.to_excel(output_filename)
    
    
print("total_numbers", total_numbers)

parsing 0 ../data/res\generali\tabula-Generali_2019_EN_short-0.csv
(1, 1)


,Solvency II value
*0,
Assets,NaN
Intangible assets,0.0
Deferred tax assets,0.0
Pension benefit surplus,0.0
"Property, plant & equipment held for own use",877909.0
Investments (other than assets held for index-linked and unit-linked contracts),68036488.0
Property (other than for own use),316964.0
"Holdings in related undertakings, including participations",17794170.0
Equities,2086266.0


parsing 1 ../data/res\generali\tabula-Generali_2019_EN_short-1.csv
(0, 1)


,1
0,
Technical provisions - non-life,17087523.0
Technical provisions - non-life (excluding health),15157230.0
TP calculated as a whole,0.0
Best estimate,14449245.0
Risk margin,707985.0
Technical provisions - health (similar to non-life),1930293.0
TP calculated as a whole,0.0
Best estimate,1904108.0
Risk margin,26185.0


parsing 2 ../data/res\generali\tabula-Generali_2019_EN_short-10.csv
(3, 2)


...

parsing 48 ../data/res\kooperativa\tabula-Kooperativa_2020_CZ_short-8.csv
(1, 1)


,C0020,C0030,C0040,C0050,C0060,C0070,C0080,C0090,C0100
*0,,,,,,,,,
R0320,8243.0,2357324.0,0.0,6680858.0,1399687.0,10839.0,3986224.0,3191047.0,702.0
R0330,-168.0,1640203.0,0.0,3147238.0,18554.0,-2712.0,1696409.0,886204.0,NaN
R0340,8411.0,717121.0,0.0,3533620.0,1381133.0,13551.0,2289815.0,2304843.0,702.0


parsing 49 ../data/res\kooperativa\tabula-Kooperativa_2020_CZ_short-9.csv
(1, 1)


,C0110,C0120,C0130,C0140,C0150,C0160,C0170,C0180
*0,,,,,,,,
R0010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R0050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R0060,-99.0,82096.0,66977.0,0.0,0.0,0.0,0.0,2520329.0
R0140,0.0,0.0,41225.0,0.0,0.0,0.0,0.0,802155.0
R0150,-99.0,82096.0,25752.0,0.0,0.0,0.0,0.0,1718174.0
R0160,1019.0,52063.0,442664.0,NaN,NaN,NaN,NaN,14530354.0
R0240,0.0,-1123.0,400845.0,0.0,0.0,0.0,0.0,7024521.0
R0250,1019.0,53186.0,41819.0,0.0,0.0,0.0,0.0,7505833.0
R0260,920.0,134159.0,509640.0,0.0,0.0,0.0,0.0,17050682.0


total_numbers 1069


In [155]:
def to_float(s):
    try:
        return float(s)
    except ValueError:
        return s


def dataframe_to_couples(df):
    dfd = df.to_dict()
    lst = []
    for k, v in dfd.items():
        kl = k.lower().split(" ")
        for ki, vi in v.items():
            kil = ki.lower().split(" ")
            lst.append((set(kl).union(set(kil)), to_float(vi), k + " " + ki))
    return lst 


def mapping_table_to_words(df):
    transform = lambda x: [x[c].lower().split(" ") if isinstance(x[c], str) else "" for c in x.index]
    df["words"] = df.apply(transform, axis=1)
    df["words"] = df.apply(lambda x: set(b for a in x["words"] for b in a), axis=1)
    return df


def mapping_match(words, mapped):
    return len(mapped.intersection(set(words)))
    

df = pd.read_excel(f"../data/mapping_table.xlsx", header=0)
words = mapping_table_to_words(df)
words["best_match_val"] = -1
words["match"] = ""
words["mapping"] = ""
# display(words)
    
for i, f in enumerate(glob.glob("../data/nice/**/*koo*.xlsx", recursive=True)):
    f = pathlib.Path(f)    
    df = pd.read_excel(f, header=0)
    df = df.set_index(df.columns[0])
#     display(df)
    couples = dataframe_to_couples(df)
    for couple in couples:
        words["mapping_match"] = words.apply(lambda x: mapping_match(x["words"], couple[0]), axis=1)
        best_idx = words["mapping_match"].idxmax()
        best_val = words["mapping_match"].max()
        if best_val > words.loc[best_idx, "best_match_val"] and best_val >= 2:
            words.at[best_idx, "match"] = couple[2]
            words.at[best_idx, "mapping"] = couple[1]
            words.at[best_idx, "best_match_val"] = best_val

display(words)
del words["words"]
del words["mapping_match"]
words.to_excel("../data/mapping_result.xlsx")

,S-code,C-code,R-code,Common,Text,Custom,words,best_match_val,match,mapping,mapping_match
0,QRT_S.02.01,C0010,R0040,NaN,Deferred tax assets,NaN,"{r0040, qrt_s.02.01, assets, tax, c0010, defer...",2,C0010 R0040,0,0
1,QRT_S.02.01,C0010,R0070,NaN,Investments (other than assets held for index-...,NaN,"{for, held, qrt_s.02.01, investments, (other, ...",2,C0010 R0070,7.63801e+07,0
2,QRT_S.02.01,C0010,R0080,NaN,Property (other than for own use),NaN,"{for, qrt_s.02.01, (other, r0080, c0010, than,...",2,C0010 R0080,0,0
3,QRT_S.02.01,C0010,R0090,NaN,"Holdings in related undertakings, including pa...",NaN,"{including, qrt_s.02.01, in, r0090, holdings, ...",2,C0010 R0090,1.15671e+07,0
4,QRT_S.02.01,C0010,R0100,NaN,Equities,NaN,"{c0010, r0100, qrt_s.02.01, equities}",2,C0010 R0100,304504,0
...,...,...,...,...,...,...,...,...,...,...,...
183,QRT_S.05.01.02,C0240,R1800,Changes in other technical provisions,Other life insurance,life,"{other, technical, provisions, in, insurance, ...",2,C0240 R1800,-222523,0
184,QRT_S.05.01.02,C0250,R1800,Changes in other technical provisions,Annuities stemming from non-life insurance con...,life,"{, obligations, c0250, other, technical, life,...",2,C0250 R1800,NaN,0
185,QRT_S.05.01.02,C0260,R1800,Changes in other technical provisions,Annuities stemming from non-life insurance con...,life,"{obligations, other, technical, life, provisio...",2,C0260 R1800,NaN,0
186,QRT_S.05.01.02,C0270,R1800,Changes in other technical provisions,Health reinsurance,life,"{other, technical, provisions, in, changes, c0...",2,C0270 R1800,NaN,0
